In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

import plotnine
from plotnine import *
plotnine.options.figure_size = (18,9)


def create_climbers_df(df, lr_model, pmf_model):
    climbers_df = (
        df
        .groupby('Name')
        .agg({'Status': 'mean', 'Problem_ID': 'size'})
        .rename(columns = {'Status': "success", 'Problem_ID': 'size'})
    )

    lr_climber_names = lr_model.climber_vocab.get_itos()[1:]
    lr_climbers = pd.DataFrame(
        {"coefs": lr_model.lr.coef_.flatten().tolist()[1:len(lr_climber_names) + 1]},
        index = lr_climber_names
    )

    pmf_climber_names = pmf_model.climber_vocab.get_itos()[1:]
    pmf_embeddings = pmf_model.climber_embedding.weight[1:].detach().numpy()
    embeddings_df = pd.DataFrame(
        pmf_embeddings,
        index = pmf_climber_names,
        columns = [f"weight_{i+1}" for i in range(pmf_embeddings.shape[1])]
    )

    climbers_df = pd.concat([climbers_df.loc[pmf_climber_names], lr_climbers.loc[pmf_climber_names], embeddings_df], axis = 1)

    ### Create PCs
    components = min(pmf_embeddings.shape)
    pca = PCA(n_components = components)
    pcs = pca.fit_transform(pmf_embeddings)
    for pc in range(components):
        climbers_df[f'PC{pc+1}'] = pcs[:, pc]

    return climbers_df


def create_correlation_matrices(dfs, pca=False):
    for replacement_level in REPLACEMENT_LEVELS:

        fig, axs = plt.subplots(nrows=1, ncols=len(LATENT_FACTORS), figsize=(28, 8))

        for num_factors in LATENT_FACTORS:
            df = dfs[f'{replacement_level}_{num_factors}']

            prefix = 'PC' if pca else 'weight'
            rows = [row for row in df.columns if row.startswith(prefix)]
            cols = ['coefs', 'size', 'success']
            df_corr = df.corr().loc[rows, cols]
            sns.heatmap(df_corr, annot=True, cmap='coolwarm', center=0, vmin=-1, vmax=1, ax=axs[num_factors-1])

        plt.suptitle(f'Correlation Matrices at RL: {replacement_level}', fontsize=16)
        plt.tight_layout()

        filename = f'results/climber_embeddings/{"corr_pca" if pca else "corr_raw"}/{replacement_level}.png'
        plt.savefig(filename)
        plt.close(fig)


def create_pc_scatter_plot(climbers_df, variable = 'success'):
    return (
        climbers_df
        .pipe(ggplot, aes(x = 'PC1', y = 'PC2', color = variable)) +
        geom_point() +
        labs(x = 'Principal Component 1', y = 'Principal Component 2', title = 'Climber Embeddings PCA Analysis') +
        theme_bw()
    )


def create_problems_df(df, pmf_model):
    problems_df = (
        df
        .groupby('Problem_ID')
        .agg({'Status': 'mean', 'Problem_ID': 'size', 'Problem': 'first', 'Year': 'first'})
        .rename(columns = {'Status': "success", 'Problem_ID': 'size'})
        .assign(category = lambda d: d['Problem'].str[:-1])
    )

    pmf_problem_names = pmf_model.problem_vocab.get_itos()[1:]
    pmf_embeddings = pmf_model.problem_embedding.weight[1:].detach().numpy()
    embeddings_df = pd.DataFrame(
        pmf_embeddings,
        index = pmf_problem_names,
        columns = [f"weight_{i+1}" for i in range(pmf_embeddings.shape[1])]
    )

    problems_df = pd.concat([problems_df.loc[pmf_problem_names], embeddings_df], axis = 1)

    ### Create PCs
    components = min(pmf_embeddings.shape)
    pca = PCA(n_components = components)
    pcs = pca.fit_transform(pmf_embeddings)
    for pc in range(components):
        problems_df[f'PC{pc+1}'] = pcs[:, pc]

    return problems_df

In [17]:
# if "__name__" == "__main__":
import torch
import pickle
from pmf import PMF
from lr import LogReg

REPLACEMENT_LEVELS = [25, 50, 100, 250, 500, 1000]
LATENT_FACTORS = [1, 2, 3, 4]

df = pd.read_csv('data/men_data.csv')

problem_dfs, climber_dfs = {}, {}
for replacement_level in REPLACEMENT_LEVELS:
    for num_factors in LATENT_FACTORS:

        with open(f"models/lr/model_rl_{replacement_level}_full_data.pkl", 'rb') as f:
            lr_model = pickle.load(f)

        for num_factors in LATENT_FACTORS:
            pmf_model = torch.load(f"models/pmf/model_rl_{replacement_level}_d_{num_factors}_full_data.pth")
            pmf_model.eval()

            climbers_df = create_climbers_df(df, lr_model, pmf_model)
            climber_dfs[f'{replacement_level}_{num_factors}'] = climbers_df

            if num_factors > 1:
                for column in ['success']:
                    p = create_pc_scatter_plot(climbers_df, column)
                    p.save(f'results/climber_embeddings/scatter_plot/{column}/PCA_{replacement_level}_{num_factors}.png')

            pmf_model = torch.load(f"models/pmf/model_rl_{replacement_level}_d_{num_factors}_full_data.pth")
            pmf_model.eval()

            problems_df = create_problems_df(df, pmf_model)
            problem_dfs[f'{replacement_level}_{num_factors}'] = problems_df

            if num_factors > 1:
                for column in ['success', 'category', 'Year']:
                    p = create_pc_scatter_plot(problems_df, column)
                    p.save(f'results/problem_embeddings/scatter_plot/{column}/PCA_{replacement_level}_{num_factors}')

                # Fit using ClimberPCs, apply to Problem PCs
                pca = PCA(n_components=2)
                climber_pcs = pca.fit_transform(climbers_df[['weight_1', 'weight_2']])
                problem_pcs = pca.transform(problems_df[['weight_1', 'weight_2']])

                problems_df['PC1'] = problem_pcs[:, 0]
                problems_df['PC2'] = problem_pcs[:, 1]

                for column in ['success', 'category', 'Year']:
                    p = create_pc_scatter_plot(problems_df, column)
                    p.save(f'results/cross_embeddings/scatter_plot/{column}/PCA_{replacement_level}_{num_factors}')

create_correlation_matrices(climber_dfs)
create_correlation_matrices(climber_dfs, pca=True)



/Users/zekeweng/.pyenv/versions/3.9.10/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.5.1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
/Users/zekeweng/.pyenv/versions/3.9.10/lib/python3.9/site-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 18 x 9 in image.
/Users/zekeweng/.pyenv/versions/3.9.10/lib/python3.9/site-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: results/climber_embeddings/scatter_plot/success/PCA_25_2.png
/Users/zekeweng/.pyenv/versions/3.9.10/lib/python3.9/site-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 18 x 9 in image.
/Users/zekeweng/.pyenv/versions/3.9.10/lib/python3.9/site-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: results/problem_embeddings/scatt

In [16]:
for replacement_level in REPLACEMENT_LEVELS:
    for num_factors in LATENT_FACTORS:
        if num_factors > 1:
            for column in ['success', 'category', 'Year']:

                # Fit using ClimberPCs, apply to Problem PCs
                pca = PCA(n_components=2)
                climber_pcs = pca.fit_transform(climber_dfs[f'{replacement_level}_{num_factors}'][['weight_1', 'weight_2']])
                problem_pcs = pca.transform(problem_dfs[f'{replacement_level}_{num_factors}'][['weight_1', 'weight_2']])

                problem_dfs[f'{replacement_level}_{num_factors}']['PC1'] = problem_pcs[:, 0]
                problem_dfs[f'{replacement_level}_{num_factors}']['PC2'] = problem_pcs[:, 1]

                for column in ['success', 'category', 'Year']:
                    p = create_pc_scatter_plot(problem_dfs[f'{replacement_level}_{num_factors}'], column)
                    p.save(f'results/cross_embeddings/scatter_plot/{column}/PCA_{replacement_level}_{num_factors}')


/Users/zekeweng/.pyenv/versions/3.9.10/lib/python3.9/site-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 18 x 9 in image.
/Users/zekeweng/.pyenv/versions/3.9.10/lib/python3.9/site-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: results/cross_embeddings/scatter_plot/success/PCA_25_2
/Users/zekeweng/.pyenv/versions/3.9.10/lib/python3.9/site-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 18 x 9 in image.
/Users/zekeweng/.pyenv/versions/3.9.10/lib/python3.9/site-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: results/cross_embeddings/scatter_plot/category/PCA_25_2
/Users/zekeweng/.pyenv/versions/3.9.10/lib/python3.9/site-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 18 x 9 in image.
/Users/zekeweng/.pyenv/versions/3.9.10/lib/python3.9/site-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: results/cross_embeddings/scatter_plot/Year/PCA_25_2
/Users/zekeweng/.pyenv/versions/3.9.10/lib/python3.9/site-packages/plotnine/ggplot.py:6